If you're not running in Saturn Cloud, you need to install these libraries:

Make sure you use the latest versions

```
pip install -U transformers accelerate bitsandbytes
```

In [12]:
pip install -U transformers accelerate bitsandbytes ipywidgets sentencepiece


  Obtaining dependency information for sentencepiece from https://files.pythonhosted.org/packages/de/42/ae30952c4a0bd773e90c9bf2579f5533037c886dfc8ec68133d5694f4dd2/sentencepiece-0.2.0-cp311-cp311-macosx_11_0_arm64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 10.7 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.2.1 -> 24.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
os.environ['HF_HOME'] = '/tmp/cache/'

In [3]:
!rm -f minsearch.py
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

--2024-06-24 23:39:05--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Herleiden van raw.githubusercontent.com (raw.githubusercontent.com)... 100.64.1.215
Verbinding maken met raw.githubusercontent.com (raw.githubusercontent.com)|100.64.1.215|:443... verbonden.
HTTP-verzoek is verzonden; wachten op antwoord... 200 OK
Lengte: 3832 (3,7K) [text/plain]
Wordt opgeslagen als: ‘minsearch.py’

minsearch.py        100%[===================>]   3,74K  --.-KB/s    in 0s      

2024-06-24 23:39:05 (32,6 MB/s) - '‘minsearch.py’' opgeslagen [3832/3832]



In [7]:
import requests 
import minsearch

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

In [8]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [9]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [ ]:
!df -h

In [2]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

In [3]:
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-xl")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-xl", device_map="auto")

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

model-00002-of-00002.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

Error while downloading from https://cdn-lfs.huggingface.co/repos/16/41/16418edd56a7c42307a0f361531c01ee227a92a98628972bd433062c276dad7c/c0c677ddeb21009b6efd97146f37fc3a0396707fb5e63ade7aff64884dce9806?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27model-00002-of-00002.safetensors%3B+filename%3D%22model-00002-of-00002.safetensors%22%3B&Expires=1719525477&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxOTUyNTQ3N319LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy8xNi80MS8xNjQxOGVkZDU2YTdjNDIzMDdhMGYzNjE1MzFjMDFlZTIyN2E5MmE5ODYyODk3MmJkNDMzMDYyYzI3NmRhZDdjL2MwYzY3N2RkZWIyMTAwOWI2ZWZkOTcxNDZmMzdmYzNhMDM5NjcwN2ZiNWU2M2FkZTdhZmY2NDg4NGRjZTk4MDY%7EcmVzcG9uc2UtY29udGVudC1kaXNwb3NpdGlvbj0qIn1dfQ__&Signature=nKpsCWUc4j61jvtHytxltMmbsSS5L%7EoR5Uc9OAOPVCoVmaj0bTTGGOHa2MZXX%7E-0ZQzi-FAFcA4MBEeFee2JRST0mqEEEUu3lVbJByuF3raoVpSX7nxBJhwg36XhRgs%7EdMswu%7E3OLjkT5q2mlmCgl7ltaezXHLvdWULi9sGnZNBMUvqrHF2zFclrgeJDImSGnUeSgBqAEak7uztu

model-00002-of-00002.safetensors:  92%|#########1| 1.79G/1.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [30]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

def llm(prompt):
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(model.device)
    attention_mask = tokenizer(prompt, return_tensors="pt").attention_mask.to(model.device)
    outputs = model.generate(input_ids, attention_mask=attention_mask)
    result = tokenizer.decode(outputs[0])
    return result

In [33]:
def llm(prompt, generate_params=None):
    if generate_params is None:
        generate_params = {}

    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(model.device)
    attention_mask = tokenizer(prompt, return_tensors="pt").attention_mask.to(model.device)
    outputs = model.generate(
        input_ids,
        max_length=generate_params.get("max_length", 100),
        num_beams=generate_params.get("num_beams", 5),
        do_sample=generate_params.get("do_sample", False),
        temperature=generate_params.get("temperature", 1.0),
        top_k=generate_params.get("top_k", 50),
        top_p=generate_params.get("top_p", 0.95),
        attention_mask=attention_mask
    )
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return result


In [34]:
rag("I just discovered the course. Can I still join it?")

/Users/vrabinin/Documents/Github/llm-zoomcamp/.venv/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:520: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


"Yes, even if you don't register, you're still eligible to submit the homeworks. Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute."

In [32]:
llm("I just discovered the course. Can I still join it?")

'<pad> Yes, you can.</s>'